# Transform orders data - Explode arrays
1. Access elements from the json object
2. Deduplicate array elements
3. Explode arrays
4. Write the transformed data to Silver schema

In [0]:
%sql 
SELECT *
FROM gizmobox.silver.orders_json;

## 1. Access elements from the JSON object

In [0]:
%sql
select
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  json_value.items
from gizmobox.silver.orders_json;

## 2. Deduplicate array elements

[array_distinct](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/array_distinct) function

In [0]:
%sql
select
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  array_distinct(json_value.items) as items
from gizmobox.silver.orders_json;

## 3. Explode arrays
[explode](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/explode) function

In [0]:
%sql
create or replace temp view tv_orders_exploded
as
select
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) as item
from gizmobox.silver.orders_json;

In [0]:
%sql
select
  order_id,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
from tv_orders_exploded;

## 4. Write the transformed data to silver schema

In [0]:
%sql
create or replace table gizmobox.silver.orders
as
select
  order_id,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
from tv_orders_exploded;

In [0]:
%sql
select
  * 
from gizmobox.silver.orders; 